# Network Analysis-Assignment 2
#### James Mallen, 19346861

#### For my data, I will construct, and characterise a pre-collected set of data from the RUGBY list and create the following

- Follower Network
- Mention Network
- Reply Network
- User-hastag network
- Hashtag co-occurrence network

I straight away just took all my libraries from assignment 1 as it is likely I will need them once again.

For some reason nx was giving an odd error when code should have been running this issue seems to be a new one 
https://github.com/microsoft/CyberBattleSim/issues/4 is where I found the solution it may require the user to restart their kernal + output

In [1]:
pip install decorator==4.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
import os.path
import numpy as np
from networkx import Graph
import pandas as pd
import matplotlib
from typing import Union
import json
import itertools
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline


## Follower Network

#### This is a network of a csv file where it is a network of who follows eachother in the rugby twitter space

In [3]:
# create the network using nx so I can start

#used below https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.edgelist.write_edgelist.html
Follower= nx.read_edgelist("friendships.csv" , delimiter = "\t", create_using=nx.DiGraph)


### I have now created a very simple direct and unweighted  network
- It is unweighted as one account as a follower is not seen as more valuable than another
- Direct as we need to know if people are following EACH-other as we all know a following on social media works 2 ways 
- Strongly connected networks are ones that if every vertex is reachable from every other vertex
- Network density refers to a measure of the prevalence of dyadic linkage or direct tie within a social network, In this case we are at 67.5% density which means that 587 is 67.5% of the total possible edges

In [4]:
print("Number of nodes:",Follower.number_of_nodes())


Number of nodes: 30


There is 30 different Twitter accounts in this network

In [5]:
 print("Number of edges",Follower.number_of_edges())

Number of edges 587


There is 587 "accounts" of accounts following other profiles

In [6]:
print(nx.is_strongly_connected(Follower))

True


In [7]:
print("%3f" %nx.density(Follower))

0.674713


Defining below will save a lot of time as I can see In Degree being used a lot throughout the project
- in Degree is the number of nodes coming into a node and out degree is simply the opposite

In [8]:
def inDegree(net: nx.DiGraph, weight:str = None) -> pd.DataFrame:
    in_degrees = pd.Series(dict(net.in_degree(weight=weight)),name="In Degree").sort_values(ascending=False)
    return pd.DataFrame(in_degrees)


indegree1 = inDegree(Follower)
indegree1.head()

,In Degree
worldrugby,27
championscup,27
englandrugby,25
rugbyworldcup,25
sixnationsrugby,24


In [9]:
def outDegree(net: nx.DiGraph, weight:str = None) -> pd.DataFrame:
    out_degrees = pd.Series(dict(net.out_degree(weight=weight)),name="Out Degree").sort_values(ascending=False)
    return pd.DataFrame(out_degrees)


outdegree1 = outDegree(Follower)
outdegree1.head()

,Out Degree
championscup,29
exeterchiefs,28
urcofficial,26
racing92,25
englandrugby,24


In [10]:
print("Average Clustering Coefficient:",nx.average_clustering(Follower))


Average Clustering Coefficient: 0.73357187195334


In [11]:
Follower=pd.DataFrame.from_dict(nx.clustering(Follower), orient='index',columns = ['ClusteringMeasure'])
Follower.sort_values(by='ClusteringMeasure', ascending=False)


,ClusteringMeasure
womens6nations,0.889655
saintsrugby,0.817187
saracens,0.800546
rugby_ie,0.768559
scotlandteam,0.767906
irishrugby,0.763434
harlequins,0.758721
bristolbears,0.750812
munsterrugby,0.746998
federugby,0.746050


#### I will now make the network as it can be much easier to work with data when you can see it laid out infront of you

In [12]:
nodeLabels=dict(zip(Follower.nodes(),Follower.nodes()))
# use zip to bind the nodes so it can be acessed as a dict
#https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html

AttributeError: 'DataFrame' object has no attribute 'nodes'

In [ ]:
plt.figure(figsize=(20, 10))
position = nx.spring_layout(Follower,k=3)
nx.draw(Follower, position,  node_size=1000,  node_color="lightblue", edge_color="grey", arrowsize=10)
nx.draw_networkx_labels(Follower, position, nodeLabels,font_weight="bold",font_size=12)
plt.title(label = "Follower Network", fontsize = 30)
plt.show()

#### Now with the graph in front of me I can see that it is teams from all around the world and I would like to be able to show this in a graph
- organisations & competitions such as skysportsrugby or worldrugby will be given their own colour if not associated with a location i.e rugby_ie
- https://python-graph-gallery.com/324-map-a-color-to-network-nodes for refrence

In [ ]:
#First we need to make a function for the graph to map each account to a country 
# g for use in a graph and of course we will start with ulsterrugby as we tend not to come first in anything else....

def countrymap(g):
    ireland = ['ulsterrugby', 'connachtrugby', 'munsterrugby','leinsterrugby', 'irishrugby', 'rugby_ie']
    scotland = ['scotlandteam']
    wales = ['welshrugbyunion']
    italy = ['federugby']
    france = ['sfparisrugby','mhr_rugby','stadetoulousain','asmofficial','rctofficial','racing92','france_rugby']
    freeagent= ['rugbyworldcup','skysportsrugby','sixnationsrugby','premrugby','worldrugby','womens6nations','championscup','urcofficial']
    england =['saracens','englandrugby','exterchiefs','bristolbears','harlequins','saintsrugby']
    
    for node in g.nodes():
        if node in ireland:
            g.nodes[node]["Country"]="Ireland"
        elif node in scotland:
            g.nodes[node]["Country"]="Scotland"
        elif node in italy:
            g.nodes[node]["Country"]="Italy"
        elif node in france:
            g.nodes[node]["Country"]="France"
        elif node in freeagent:
            g.nodes[node]["Country"]="Org/Comp"
        else:
            g.nodes[node]["Country"]="England"
countrymap(Follower)

#map colours to the country
colourmap = {"Ireland":"green","Scotland":"blue","Wales":"red","Italy":"purple","France":"yellow","Org/Comp":"brown","England":"orange"}
colour_map = []
for node in Follower.nodes():
    colour_map.append(colourmap[Follower.nodes[node]["Country"]])


- Ireland = Green
- Scotland = Blue
- Wales = Red
- Italy = Purple
- France = Yellow
- England = Orange
- Org/Comp = Brown


In [ ]:
plt.figure(figsize=(20, 10))
position = nx.spring_layout(Follower, k = 3)

nx.draw(Follower, position, node_color=colour_map, node_size=1000, edge_color="grey", arrowsize=10)
nx.draw_networkx_labels(Follower, position, nodeLabels, font_size=10, font_weight="bold")

plt.title(label = "Follower Network(Colour Coded)", fontsize = 20)

plt.show()

# Degree of Centrality
- Measures the amount of nodes pointing towards a node i.e follower count of only nodes in this system
- Video followed https://www.youtube.com/watch?v=XMDxpIXh-as

## Mention Network 
- Task 2 

In [ ]:
json_file1 = open("rugby/tweets.jsonl","r")

# accesses the file tweets

with open('rugby/tweets.jsonl') as f:
    tweets_df = pd.DataFrame(json.loads(line.strip()) for line in f)

tweets_df.columns # This will give us all the 'id's of every piece of data in the json file


In [ ]:
display(tweets_df.head(2)) # example of an entry


Now since there is a large amount of data entries I will drop a lot of them to make it easier on me to deal with


In [ ]:
tweets_df=tweets_df.drop( ['quoted_status_id_str', 'quoted_status_permalink','quoted_status', 'possibly_sensitive', 'lang','quoted_status_id',
                           'is_quote_status','geo', 'coordinates','in_reply_to_status_id_str', 'source',
                           'truncated','display_text_range','id_str','created_at'],axis =1)


In [ ]:
tweets_df.columns
#double check the columns have been removed

In [ ]:
display(tweets_df.head(5)) # example of an entry


I now have the issue of not knowing the name of the users but below will extract that info

In [ ]:
mention_df = tweets_df[['user', 'entities']].copy()

mention_df['user_mentions'] = mention_df['entities'].apply(
    lambda e: e['user_mentions'])

mention_df.drop(['entities'], axis=1, inplace=True)
mention_df = mention_df[mention_df['user_mentions'].map(len) != 0]

mention_df['user_mentions'] = mention_df['user_mentions'].apply(
    lambda x: [u['screen_name'].lower() for u in x])

mention_df['user'] = mention_df['user'].apply(
    lambda u: u['screen_name'].lower())

mention_df

.Explode is a handy function which lets us see how many times a user has mentioned another user in one of their tweets

In [ ]:
mention_df = mention_df.explode('user_mentions')
mention_df = pd.DataFrame(mention_df.value_counts(), columns=['number_mentions'])
mention_df.reset_index(inplace=True)
mention_df.head(10)

We can now create a weighted directed network like above

In [ ]:
mention_net = nx.from_pandas_edgelist(mention_df,  target='user_mentions'
                                      ,source='user',edge_attr='number_mentions',create_using=nx.DiGraph)

In [ ]:
print("Number of nodes",mention_net.number_of_nodes())

In [ ]:
print("Number of edges",mention_net.number_of_edges()) 

In [ ]:
print(nx.is_strongly_connected(mention_net))

- Strongly connected networks are ones that if every vertex is reachable from every other vertex, in this case the network is not strongly connected


In [ ]:
print("%3f" %nx.density(mention_net))

In [ ]:
print("Number of strongly connected components:",nx.number_strongly_connected_components(mention_net))


In [ ]:
print("Number of weakly connected components: ", nx.number_weakly_connected_components(mention_net))


In [ ]:
print("Density %.10f" % nx.density(mention_net))


In [ ]:
indegree2 = inDegree(mention_net)
indegree2.head()

In [ ]:
outdegree2 = outDegree(mention_net)
outdegree2.head()

Compared to the last network we can see a huge diffrence.
- This network is much larger with much less density of connected nodes
- Think of the last network being a group of friends and who they would call friends and this network is an entire college's friend network
- Only 0.0006% accounts mention eachother in this network which is extremely low

#### Clustering Co-Efficent
- Can easily reuse the code from assignment 1
- Average clustering would be a better measure to use as there is manny points with a cluster score of 0 which can throw off measures

In [ ]:
print("Average Clustering Coefficient:",nx.average_clustering(mention_net))


The local clustering of each node in mention_net is the 
fraction of triangles that actually exist over 
all possible triangles in its neighborhood.
The average clustering coefficient of a graph 
mention_net is the mean of local clusterings.
- https://www.geeksforgeeks.org/clustering-coefficient-graph-theory/#:~:text=The%20local%20clustering%20of%20each,the%20mean%20of%20local%20clusterings.
  

As we can see this network is far greater than the csv file we had in task1

#### Plotting distribution of in_degree 

plt.figure(figsize=(20, 10))
sn.histplot(data=indegree2,color='lightblue', x ='In Degree', y= 'Occurrence')
plt.xlabel('In Degree',size= 10)
plt.ylabel('Occurrence',size=10)
plt.title(label = "Distribution of In Degree Across All Nodes", fontsize = 20)

plt.show()

## Reply Network 
- Task 3 

## User-hashtag Network 
- Task 4

## Hash-tag Co-occurrence Network 
- Task 5 